In [1]:
import boto3
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'Fall_2021/In_Class_Assignments/framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Defining the input and target variables 
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']
## Splitting the data into training and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [3]:
## Adaboost
md1 = AdaBoostClassifier(base_estimator =  DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)

## Extracting feature importance
md1.feature_importances_

array([0.01709619, 0.10941846, 0.01473028, 0.18422556, 0.17876445,
       0.24039038, 0.11385496, 0.14151973])

In [4]:
## Adaboost with top 5 important variables
X_train_ada = X_train[['totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
X_test_ada = X_test[['totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]

md1_top5 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train_ada, Y_train)

## Predicting on the test dataset
pred1 = md1_top5.predict_proba(X_test_ada)[:, 1]
pred1 = np.where(pred1 < 0.1, 0, 1)

## Compute the recall
recall_score(Y_test, pred1)

1.0

In [5]:
## Random forest
md2 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

## Extracting the feature importance
md2.feature_importances_

array([0.03898882, 0.27227299, 0.00610209, 0.07788113, 0.33183572,
       0.07251915, 0.03205945, 0.16834064])

In [6]:
## Random forest top 5 important variables
X_train_rf = X_train[['age', 'totChol', 'BMI', 'sysBP', 'glucose']]
X_test_rf = X_test[['age', 'totChol', 'BMI', 'sysBP', 'glucose']]

md2_top5 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train_rf, Y_train)

## Predicting on the test dataset
pred2 = md2_top5.predict_proba(X_test_rf)[:, 1]
pred2 = np.where(pred2 < 0.1, 0, 1)

## computing the recall
recall_score(Y_test, pred2)

0.8839285714285714